## Azure 인증 설정

In [1]:
import os
from azure.identity import AzureCliCredential

# ============================================
# 🔐 인증 방식 선택 (둘 중 하나만 True로 설정!)
# ============================================
USE_API_KEY = True           # API Key 기반 인증
USE_CLI_CREDENTIAL = False   # AzureCliCredential 사용 (az login 필요)

# ============================================

if USE_API_KEY and USE_CLI_CREDENTIAL:
    raise ValueError("❌ 둘 중 하나만 True로 설정하세요!")

if not USE_API_KEY and not USE_CLI_CREDENTIAL:
    raise ValueError("❌ 최소 하나는 True로 설정하세요!")

if USE_API_KEY:
    # API Key 인증 - 아래 값을 본인 정보로 수정!
    AZURE_OPENAI_API_KEY = os.environ.get("AZURE_OPENAI_API_KEY", "").strip()  # 👈 API Key 입력
    
    credential = None  # API Key 사용 시 credential 불필요
    os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
    
    print("🔐 API Key 인증 사용")
    print("   (API Key로 Azure OpenAI에 직접 연결)")
    
    if AZURE_OPENAI_API_KEY == "your-api-key-here":
        print("\n⚠️ API Key를 실제 값으로 수정하세요!")
    else:
        print("\n✅ API Key 설정 완료!")

elif USE_CLI_CREDENTIAL:
    credential = AzureCliCredential()
    print("🔐 AzureCliCredential 사용")
    print("   (터미널에서 'az login' 필요)")
    
    # 인증 테스트
    try:
        token = credential.get_token("https://cognitiveservices.azure.com/.default")
        print("\n✅ Azure CLI 인증 성공!")
    except Exception as e:
        print(f"\n❌ 인증 실패: {e}")
        print("   터미널에서 'az login'을 실행하세요.")

🔐 API Key 인증 사용
   (API Key로 Azure OpenAI에 직접 연결)

✅ API Key 설정 완료!


## Azure OpenAI 설정

In [2]:
import os

#  아래 값을 본인의 Azure OpenAI 정보로 수정하세요!
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://hbjoo-mlg6s9pj-eastus2.cognitiveservices.azure.com"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "gpt-4o"  # 또는 gpt-4.1

print("✅ Azure OpenAI 설정 완료!")
print(f"   Endpoint: {os.environ['AZURE_OPENAI_ENDPOINT']}")
print(f"   Deployment: {os.environ['AZURE_OPENAI_CHAT_DEPLOYMENT_NAME']}")

✅ Azure OpenAI 설정 완료!
   Endpoint: https://hbjoo-mlg6s9pj-eastus2.cognitiveservices.azure.com
   Deployment: gpt-4o


## STEP 1️⃣ DSR 기반 최대 대출 계산

In [3]:
# 시중 은행 주택담보대출 평균 금리 조회 (한국은행 ECOS API)
# DATA/mortgage_rate.csv에 해당 월 금리가 있으면 사용, 없을 때만 API 호출 후 CSV에 저장
# 환경변수 BOK_ECOS_API_KEY 설정 시 API 사용, 미설정 시 기본값 4.48% 사용
import os
import csv
from datetime import date, datetime
from pathlib import Path
from dotenv import load_dotenv

# .env를 cwd부터 상위 폴더로 찾아서 로드 (노트북 cwd가 프로젝트 루트가 아닐 수 있음)
_cwd = Path.cwd().resolve()
for _ in range(6):
    _env = _cwd / ".env"
    if _env.exists():
        load_dotenv(_env)
        break
    if _cwd.parent == _cwd:
        break
    _cwd = _cwd.parent
else:
    load_dotenv()
print("BOK_ECOS_API_KEY:", "설정됨" if os.environ.get("BOK_ECOS_API_KEY", "").strip() else "미설정 (프로젝트 루트 .env 확인)")

_DEFAULT_RATE = 0.0448  # API 미사용 시 적용 (연 4.48%)

def _csv_path():
    return Path("DATA") / "mortgage_rate.csv"

def _minus_months(d, months):
    """날짜에서 months개월 뺀 날짜 반환 (월 단위)"""
    y, m = d.year, d.month
    m -= months
    while m <= 0:
        m += 12
        y -= 1
    return date(y, m, 1)

def _save_rate_to_csv(csv_path, yyyymm, rate):
    """해당 월 금리를 CSV에 한 줄 추가"""
    try:
        csv_path = Path(csv_path).resolve()
        csv_path.parent.mkdir(parents=True, exist_ok=True)
        file_exists = csv_path.exists()
        need_header = not file_exists or csv_path.stat().st_size == 0
        with open(csv_path, "a", encoding="utf-8", newline="") as f:
            w = csv.DictWriter(f, fieldnames=["yyyymm", "rate"])
            if need_header:
                w.writeheader()
            w.writerow({"yyyymm": yyyymm, "rate": rate})
    except Exception:
        pass

def get_mortgage_rate(as_of_date=None):
    """
    해당 날짜 기준 예금은행 주택담보대출 금리(연%)를 소수로 반환.
    ECOS는 약 2달 지연 공표이므로, 요청일 기준 2달 전 월 데이터를 API로 조회함.
    """
    if as_of_date is None:
        as_of_date = date.today()
    if isinstance(as_of_date, datetime):
        as_of_date = as_of_date.date()
    yyyymm = as_of_date.strftime("%Y%m")
    effective_date = _minus_months(as_of_date, 2)
    effective_yyyymm = effective_date.strftime("%Y%m")
    csv_path = _csv_path().resolve()
    if csv_path.exists():
        try:
            with open(csv_path, "r", encoding="utf-8") as f:
                r = csv.DictReader(f)
                for row in r:
                    if row.get("yyyymm") == yyyymm and row.get("rate", "").strip():
                        return float(row["rate"].strip())
        except Exception:
            pass
    api_key = os.environ.get("BOK_ECOS_API_KEY", "").strip()
    rate = None
    if api_key:
        try:
            import urllib.request
            import json
            # 121Y006 + BECBLA0302: 예금은행 대출금리(신규취급액 기준) 주택담보대출, 현재일 기준 2달 전 월 조회
            url = f"https://ecos.bok.or.kr/api/StatisticSearch/{api_key}/json/kr/1/10/121Y006/M/{effective_yyyymm}/{effective_yyyymm}/BECBLA0302/"
            print("url: ",url)
            with urllib.request.urlopen(url, timeout=10) as res:
                data = json.loads(res.read().decode())
            stat = data.get("StatisticSearch", {})
            result = stat.get("RESULT", {})
            if isinstance(result, dict) and result.get("CODE") != "000":
                pass
            else:
                rows = stat.get("row") or []
                if isinstance(rows, dict):
                    rows = [rows]
                if rows:
                    val = rows[0].get("DATA_VALUE")
                    if val is not None and str(val).strip():
                        rate = float(str(val).replace(",", ".").strip()) / 100.0
        except Exception:
            pass
        if rate is None:
            print(f"[get_mortgage_rate] ECOS API 호출했으나 금리 추출 실패. 기본금리({_DEFAULT_RATE*100:.2f}%) 사용.")
    if rate is None:
        rate = _DEFAULT_RATE
    _save_rate_to_csv(csv_path, yyyymm, rate)
    return rate

BOK_ECOS_API_KEY: 설정됨


In [4]:
from typing import Literal

LoanType = Literal["mortgage", "credit", "other"]
Region = Literal["capital", "local"]  # capital=수도권, local=지방


def get_stress_rate_stage3(
    *,
    loan_type: LoanType,
    region: Region,
    is_capital_or_regulated: bool,
) -> float:
    """
    2026년 기준 스트레스 DSR 3단계 가산금리
    - 주담대: 수도권/규제지역 3.0%, 지방 1.5%
    - 신용대출: 가산 없음(0%)
    """
    if loan_type == "credit":
        return 0.0

    if loan_type == "mortgage":
        if is_capital_or_regulated or region == "capital":
            return 0.03
        return 0.015

    return 0.0


# DSR = (연간 모든 대출 원리금 상환액) / 연소득
def loan_by_dsr(
    annual_income: float,
    *,
    existing_annual_interest: float = 0.0,
    existing_annual_principal: float = 0.0,
    loan_type: LoanType = "mortgage",
    region: Region = "capital",
    is_capital_or_regulated: bool = True,
    term_years: int = 30,
):
    """
    스트레스 DSR 3단계 반영 대출 가능액 계산
    - annual_rate(연 금리)로 대출했을 때 연간 원리금 상환액이 dsr_limit_annual을 넘지 않도록 하는
      최대 원금을 반환
    - 기존 대출(연 이자 + 연 원금)은 DSR 한도에서 차감
    - 상환기간은 최대 30년
    """

    dsr_limit_annual = annual_income * 0.4

    existing_annual_total = max(existing_annual_interest, 0) + max(existing_annual_principal, 0)
    available_annual_payment = dsr_limit_annual - existing_annual_total
    if available_annual_payment <= 0:
        return 0.0

    base_r = get_mortgage_rate()
    stress_r = get_stress_rate_stage3(
        loan_type=loan_type,
        region=region,
        is_capital_or_regulated=is_capital_or_regulated,
    )
    annual_rate = base_r + stress_r

    effective_years = max(1, min(int(term_years), 30))

    # annual_rate가 0 이하면 단순 분할 상환으로 근사
    if annual_rate <= 0:
        return available_annual_payment * effective_years

    # 연 단위 원리금균등 상환 역산:
    # annual_payment = principal * [r(1+r)^n / ((1+r)^n - 1)]
    # => principal = annual_payment * [((1+r)^n - 1) / (r(1+r)^n)]
    factor = ((1 + annual_rate) ** effective_years - 1) / (
        annual_rate * (1 + annual_rate) ** effective_years
    )
    return available_annual_payment * factor


## STEP 2️⃣ 취득세 함수

In [5]:
def calculated_acquisition_tax(price: int | float) -> float:
    """취득세 계산 함수 (원 단위)"""
    if price <= 600_000_000:
        return price * 0.011   # 1.1%
    elif price <= 900_000_000:
        return price * 0.022   # 2.2%
    else:
        return price * 0.033   # 3.3%


## STEP 3️⃣ 전체 최대 매수가격 계산 (반복 탐색)

In [6]:
## 최대 금액 계산기 로직
def max_purchase_price(
    annual_income,
    cash,
    ltv=None,
    is_first_home=False,
    annual_interest_payment=0,
    annual_principal_payment=0,
    loan_type="mortgage",
    region="capital",
    is_capital_or_regulated=True,
    term_years=30,
):
    """정책(대출한도/LTV) + DSR(최대 30년 연상환 기준)을 반영한 최대 매수 가능 금액 계산"""
    loan_dsr_limit = loan_by_dsr(
        annual_income,
        existing_annual_interest=annual_interest_payment,
        existing_annual_principal=annual_principal_payment,
        loan_type=loan_type,
        region=region,
        is_capital_or_regulated=is_capital_or_regulated,
        term_years=term_years,
    )

    # 정책 대출한도: 생애최초 6억, 1주택자 2억
    policy_loan_cap = 600_000_000 if is_first_home else 200_000_000

    # LTV 정책
    # - 생애최초: 최대 70%
    # - 1주택자: 정책한도(2억) 기준으로 계산되도록 기본값 100%
    if is_first_home:
        if ltv is None:
            ltv = 0.7
        else:
            ltv = min(ltv, 0.7)
    else:
        if ltv is None:
            ltv = 1.0
        else:
            ltv = max(0.0, ltv)

    price = 10_000_000   # 시작값 1천만
    step = 10_000_000    # 1천만원 단위 탐색

    max_price = 0

    while price < 200_000_000_000:
        loan_ltv = price * ltv
        loan = min(loan_dsr_limit, loan_ltv, policy_loan_cap)

        tax = calculated_acquisition_tax(price)

        if price + tax <= loan + cash:
            max_price = price
            price += step
        else:
            break

    max_tax = int(calculated_acquisition_tax(max_price)) if max_price > 0 else 0
    return int(max_price), max_tax



## 메모리 클래스 정의

In [8]:
from agent_framework import ContextProvider, Context, ChatMessage
from collections.abc import Sequence, MutableSequence
from typing import Any
import re


class LoanInfoMemory(ContextProvider):
    """입력 정보(연소득/현금/첫구매/연이자/연원금)만 저장하는 메모리"""

    def __init__(self, chat_client):
        self._chat_client = chat_client
        self.loan_info = {
            "annual_income": None,
            "cash": None,
            "is_first_home": None,
            "annual_interest_payment": None,
            "annual_principal_payment": None,
        }

    def _parse_korean_money(self, text_fragment: str) -> int | None:
        if not text_fragment:
            return None

        frag = text_fragment.replace(",", " ")
        total = 0.0
        found_unit = False

        unit_patterns = [
            (r'(\d+(?:\.\d+)?)\s*(억|억원)', 100_000_000),
            (r'(\d+(?:\.\d+)?)\s*(천만|천만원)', 10_000_000),
            (r'(\d+(?:\.\d+)?)\s*(만|만원)', 10_000),
            (r'(\d+(?:\.\d+)?)\s*(원)', 1),
        ]

        for pattern, scale in unit_patterns:
            for m in re.finditer(pattern, frag):
                total += float(m.group(1)) * scale
                found_unit = True

        for m in re.finditer(r'(\d+(?:\.\d+)?)\s*천(?!\s*만)', frag):
            total += float(m.group(1)) * 10_000_000
            found_unit = True

        for m in re.finditer(r'(\d+(?:\.\d+)?)\s*백(?!\s*만)', frag):
            total += float(m.group(1)) * 1_000_000
            found_unit = True

        if found_unit and total > 0:
            return int(total)

        m = re.search(r'(\d+(?:\.\d+)?)', frag)
        if m:
            return int(float(m.group(1)))
        return None

    def _extract_amount(self, text: str, keywords: list[str]) -> int | None:
        """키워드 인접 금액만 추출(0 포함)"""
        text_lower = text.lower()
        money_expr = r'((?:\d+(?:\.\d+)?\s*(?:억|억원|천만|천만원|만|만원|원|천|백)\s*)+)'

        for keyword in keywords:
            keyword_lower = keyword.lower()
            if keyword_lower not in text_lower:
                continue

            # 1) 키워드 뒤 단위 포함 금액
            pattern_after_unit = rf'{keyword_lower}[^\d\n]{{0,40}}{money_expr}'
            m_after_unit = re.search(pattern_after_unit, text_lower, flags=re.IGNORECASE)
            if m_after_unit:
                amt = self._parse_korean_money(m_after_unit.group(1))
                if amt is not None:
                    return amt

            # 2) 키워드 뒤 숫자만(예: 연이자 0)
            pattern_after_num = rf'{keyword_lower}[^\d\n]{{0,40}}(\d+(?:\.\d+)?)'
            m_after_num = re.search(pattern_after_num, text_lower, flags=re.IGNORECASE)
            if m_after_num:
                return int(float(m_after_num.group(1)))

            # 3) 키워드 앞 단위 포함 금액
            pattern_before_unit = rf'{money_expr}[^\d\n]{{0,40}}{keyword_lower}'
            m_before_unit = re.search(pattern_before_unit, text_lower, flags=re.IGNORECASE)
            if m_before_unit:
                amt = self._parse_korean_money(m_before_unit.group(1))
                if amt is not None:
                    return amt

            # 4) 키워드 앞 숫자만
            pattern_before_num = rf'(\d+(?:\.\d+)?)[^\d\n]{{0,40}}{keyword_lower}'
            m_before_num = re.search(pattern_before_num, text_lower, flags=re.IGNORECASE)
            if m_before_num:
                return int(float(m_before_num.group(1)))

        return None

    def _extract_info_from_text(self, text: str) -> None:
        text_lower = text.lower().strip()
        if not text_lower:
            return

        no_loan_patterns = [
            r'보유\s*대출\s*(이|은|는)?\s*없',
            r'기존\s*대출\s*(이|은|는)?\s*없',
            r'대출\s*(이|은|는)?\s*없',
            r'대출\s*없음',
            r'무대출',
        ]
        if any(re.search(p, text_lower) for p in no_loan_patterns):
            self.loan_info["annual_interest_payment"] = 0
            self.loan_info["annual_principal_payment"] = 0

        income = self._extract_amount(text_lower, ["연소득", "연봉", "연수입", "연 수입", "연수익", "연 수익", "연간소득"])
        if income is not None:
            self.loan_info["annual_income"] = income

        cash = self._extract_amount(text_lower, ["보유현금", "보유 현금", "현금", "예금", "저축", "보유자금", "보유 자금", "자금"])
        if cash is not None:
            self.loan_info["cash"] = cash

        interest_payment = self._extract_amount(
            text_lower,
            ["연이자", "연 이자", "상환 이자", "연간 이자", "연이자상환액", "연 이자 상환액"],
        )
        if interest_payment is not None:
            self.loan_info["annual_interest_payment"] = interest_payment

        principal_payment = self._extract_amount(
            text_lower,
            ["연원금", "연 원금", "상환 원금", "연간 원금", "원금상환액", "연원금상환액", "연 권금", "연권금"],
        )
        if principal_payment is not None:
            self.loan_info["annual_principal_payment"] = principal_payment

        negated_first_home = re.search(
            r'(생애\s*최초|생애\s*첫\s*구매|첫\s*구매|첫\s*주택|처음\s*사는|생애최초구매여부).{0,12}(아니|아님|아니요|아니오|아니에요)',
            text_lower,
        )
        explicit_false = any(k in text_lower for k in ["생애최초 아님", "첫구매 아님", "1주택", "유주택", "다주택"])
        explicit_true_with_answer = re.search(
            r'(생애\s*최초|생애\s*첫\s*구매|첫\s*구매|첫\s*주택|처음\s*사는|생애최초구매여부).{0,12}(예|네|맞|true)',
            text_lower,
        )
        question_like = any(k in text_lower for k in ["?", "여부", "인가", "인가요", "맞나요", "확인"])
        explicit_true_keyword = any(
            k in text_lower for k in ["생애최초", "생애 첫 구매", "첫구매", "첫 구매", "첫 주택", "처음 사는", "무주택"]
        )

        if negated_first_home or explicit_false:
            self.loan_info["is_first_home"] = False
        elif explicit_true_with_answer:
            self.loan_info["is_first_home"] = True
        elif explicit_true_keyword and not question_like:
            self.loan_info["is_first_home"] = True

    def _extract_user_texts_from_messages(self, messages: ChatMessage | MutableSequence[ChatMessage] = None) -> list[str]:
        if not messages:
            return []

        if isinstance(messages, (list, tuple)):
            msg_list = list(messages)
        else:
            msg_list = [messages]

        user_texts: list[str] = []
        for msg in msg_list:
            role_text = ""
            role_obj = getattr(msg, "role", None)
            role_candidates = [
                role_obj,
                getattr(role_obj, "value", None),
                getattr(role_obj, "name", None),
                getattr(msg, "author_role", None),
            ]

            if isinstance(msg, dict):
                role_candidates.append(msg.get("role"))

            role_text = " ".join(str(v).lower() for v in role_candidates if v is not None).strip()
            if role_text and ("user" not in role_text and "human" not in role_text):
                continue

            contents = getattr(msg, "contents", None)
            if contents is None:
                contents = getattr(msg, "content", None)
            if contents is None and isinstance(msg, dict):
                contents = msg.get("contents", msg.get("content"))

            if contents is None:
                text_attr = getattr(msg, "text", None)
                if text_attr:
                    user_texts.append(str(text_attr).strip())
                continue

            if isinstance(contents, (str, bytes)):
                contents = [contents]
            elif not isinstance(contents, (list, tuple)):
                contents = [contents]

            text_parts = []
            for content in contents:
                if content is None:
                    continue
                if isinstance(content, str):
                    text_parts.append(content)
                    continue
                if isinstance(content, bytes):
                    text_parts.append(content.decode("utf-8", errors="ignore"))
                    continue
                if isinstance(content, dict):
                    if content.get("text") is not None:
                        text_parts.append(str(content.get("text")))
                        continue
                    if content.get("content") is not None:
                        text_parts.append(str(content.get("content")))
                        continue
                if hasattr(content, "text") and getattr(content, "text") is not None:
                    text_parts.append(str(content.text))
                    continue
                if hasattr(content, "content") and getattr(content, "content") is not None:
                    text_parts.append(str(content.content))
                    continue

            text = "".join(text_parts).strip()
            if text:
                user_texts.append(text)

        return user_texts

    def _ingest_messages(self, messages: ChatMessage | MutableSequence[ChatMessage] = None, **kwargs: Any) -> None:
        candidates = [
            messages,
            kwargs.get("request_messages"),
            kwargs.get("input_messages"),
            kwargs.get("thread_messages"),
        ]

        seen = set()
        for candidate in candidates:
            if candidate is None:
                continue
            key = id(candidate)
            if key in seen:
                continue
            seen.add(key)

            user_texts = self._extract_user_texts_from_messages(candidate)
            for text in user_texts:
                self._extract_info_from_text(text)

    def missing_required_fields(self) -> list[str]:
        required = [
            ("annual_income", "연소득"),
            ("cash", "보유현금"),
            ("is_first_home", "생애최초구매여부"),
            ("annual_interest_payment", "보유대출 연이자 상환액"),
            ("annual_principal_payment", "보유대출 연원금 상환액"),
        ]
        return [label for key, label in required if self.loan_info[key] is None]

    async def invoking(self, messages: ChatMessage | MutableSequence[ChatMessage] = None, **kwargs) -> Context | None:
        self._ingest_messages(messages=messages, **kwargs)

        info_parts = ["[loan_info 메모리]"]
        info_parts.append(f"- 연소득: {self.loan_info['annual_income'] if self.loan_info['annual_income'] is not None else '미입력'}")
        info_parts.append(f"- 보유현금: {self.loan_info['cash'] if self.loan_info['cash'] is not None else '미입력'}")
        info_parts.append(f"- 생애최초구매여부: {self.loan_info['is_first_home'] if self.loan_info['is_first_home'] is not None else '미입력'}")
        info_parts.append(f"- 보유대출 연이자 상환액: {self.loan_info['annual_interest_payment'] if self.loan_info['annual_interest_payment'] is not None else '미입력'}")
        info_parts.append(f"- 보유대출 연원금 상환액: {self.loan_info['annual_principal_payment'] if self.loan_info['annual_principal_payment'] is not None else '미입력'}")

        missing = self.missing_required_fields()
        if missing:
            info_parts.append("- 미입력 항목: " + ", ".join(missing))

        return Context(instructions="\n".join(info_parts))

    async def invoked(
        self,
        request_messages: ChatMessage | Sequence[ChatMessage],
        response_messages: ChatMessage | Sequence[ChatMessage] | None = None,
        invoke_exception: Exception | None = None,
        **kwargs: Any,
    ) -> None:
        self._ingest_messages(messages=request_messages, **kwargs)


class PurchaseResultMemory(ContextProvider):
    """loan_info 메모리를 기반으로 계산 결과(max_price/max_tax)만 저장하는 메모리"""

    def __init__(self, loan_info_memory: LoanInfoMemory):
        self.loan_info_memory = loan_info_memory
        self.result_info = {
            "max_price": None,
            "max_tax": None,
        }

    def _clear_result(self) -> None:
        self.result_info["max_price"] = None
        self.result_info["max_tax"] = None

    def _update_result(self) -> tuple[int, int] | None:
        # 방어 코드: 필수값이 하나라도 없으면 계산하지 않는다.
        missing = self.loan_info_memory.missing_required_fields()
        if missing:
            self._clear_result()
            return None

        info = self.loan_info_memory.loan_info
        max_price, max_tax = max_purchase_price(
            annual_income=info["annual_income"],
            cash=info["cash"],
            is_first_home=info["is_first_home"],
            annual_interest_payment=info["annual_interest_payment"],
            annual_principal_payment=info["annual_principal_payment"],
        )

        self.result_info["max_price"] = int(max_price)
        self.result_info["max_tax"] = int(max_tax)
        return int(max_price), int(max_tax)

    async def invoking(self, messages: ChatMessage | MutableSequence[ChatMessage] = None, **kwargs) -> Context | None:
        lines = ["[purchase_result 메모리]"]
        missing = self.loan_info_memory.missing_required_fields()

        # loan_info 5개가 모두 있을 때만 계산 함수 실행
        if missing:
            self._clear_result()
            lines.append("- 계산 상태: 대기(loan_info 필수값 부족)")
            lines.append("- 미입력 항목: " + ", ".join(missing))
            return Context(instructions="\n".join(lines))

        calc = self._update_result()
        if calc is None:
            lines.append("- 계산 상태: 대기(loan_info 필수값 부족)")
        else:
            lines.append(f"- max_price: {self.result_info['max_price']:,}원")
            lines.append(f"- max_tax: {self.result_info['max_tax']:,}원")
            lines.append("- 위 계산값은 max_purchase_price 함수 결과이며 답변에 그대로 사용")
        return Context(instructions="\n".join(lines))

    async def invoked(
        self,
        request_messages: ChatMessage | Sequence[ChatMessage],
        response_messages: ChatMessage | Sequence[ChatMessage] | None = None,
        invoke_exception: Exception | None = None,
        **kwargs: Any,
    ) -> None:
        if self.loan_info_memory.missing_required_fields():
            self._clear_result()
            return
        self._update_result()


class BudgetMemory(ContextProvider):
    """loan_info 메모리 + purchase_result 메모리를 합쳐 에이전트에 제공"""

    def __init__(self, loan_info_memory: LoanInfoMemory, purchase_result_memory: PurchaseResultMemory):
        self.loan_info_memory = loan_info_memory
        self.purchase_result_memory = purchase_result_memory

    async def invoking(self, messages: ChatMessage | MutableSequence[ChatMessage] = None, **kwargs) -> Context | None:
        c1 = await self.loan_info_memory.invoking(messages=messages, **kwargs)
        c2 = await self.purchase_result_memory.invoking(messages=messages, **kwargs)

        parts = []
        if c1 and getattr(c1, 'instructions', None):
            parts.append(c1.instructions)
        if c2 and getattr(c2, 'instructions', None):
            parts.append(c2.instructions)

        if parts:
            parts.append("\n답변 규칙: purchase_result 메모리의 max_price/max_tax를 그대로 사용하고 임의 재계산 금지.")
            return Context(instructions="\n\n".join(parts))
        return None

    async def invoked(
        self,
        request_messages: ChatMessage | Sequence[ChatMessage],
        response_messages: ChatMessage | Sequence[ChatMessage] | None = None,
        invoke_exception: Exception | None = None,
        **kwargs: Any,
    ) -> None:
        await self.loan_info_memory.invoked(
            request_messages=request_messages,
            response_messages=response_messages,
            invoke_exception=invoke_exception,
            **kwargs,
        )
        await self.purchase_result_memory.invoked(
            request_messages=request_messages,
            response_messages=response_messages,
            invoke_exception=invoke_exception,
            **kwargs,
        )


print("✅ LoanInfoMemory / PurchaseResultMemory / BudgetMemory 정의 완료")




✅ LoanInfoMemory / PurchaseResultMemory / BudgetMemory 정의 완료


## 에이전트 생성

In [9]:
from agent_framework.azure import AzureOpenAIChatClient
from types import MethodType
import re

# Budget Agent 생성
if USE_API_KEY:
    chat_client = AzureOpenAIChatClient(
        api_key=os.environ["AZURE_OPENAI_API_KEY"]
    )
else:
    chat_client = AzureOpenAIChatClient(credential=credential)

# 메모리 분리 생성
loan_info_memory = LoanInfoMemory(chat_client)
purchase_result_memory = PurchaseResultMemory(loan_info_memory)
budget_memory = BudgetMemory(loan_info_memory, purchase_result_memory)

# 하위 호환 alias
loan_memory = loan_info_memory

agent = chat_client.as_agent(
    instructions="""
    아파트 구매자의 최대 매수 가능 금액을 계산하는 어시스턴트입니다.

    절대 규칙:
    - 숫자 계산(최대 매수가/취득세)을 임의로 수행하거나 추정하지 마세요.
    - 숫자는 purchase_result 메모리의 max_price, max_tax만 사용하세요.
    - 값이 없으면 숫자를 말하지 말고 누락 항목만 질문하세요.

    1) loan_info 메모리에 아래 5개 입력을 저장하세요.
    - 연소득(부부라면 합산 연소득)
    - 보유현금
    - 생애첫구매여부
    - 보유대출 연이자 상환액(원/년)
    - 보유대출 연원금 상환액(원/년)
    (사용자가 보유대출이 없다고 말하면 두 값을 0원으로 저장)

    2) 입력 5개가 모두 저장되면 max_purchase_price 함수로 계산한 결과를
       purchase_result 메모리(max_price, max_tax)에 저장하세요.

    3) 답변은 purchase_result 메모리의 max_price / max_tax 값을 그대로 복사해 사용하세요.
       임의 재계산/재해석/반올림 금지.
    """,
    name="BudgetAgent",
    context_provider=budget_memory
)

# ===== 출력 강제 가드 =====
_RESULT_KEYWORDS = (
    "최대", "매수", "구매", "아파트 가격", "취득세", "max_price", "max_tax", "얼마"
)
_MEMORY_KEYWORDS = (
    "메모리", "저장된 값", "저장 값", "현재 값", "상태", "loan_info", "purchase_result"
)


def _is_result_request(user_text: str) -> bool:
    txt = (user_text or "").strip().lower()
    if not txt:
        return False
    return any(k in txt for k in _RESULT_KEYWORDS)


def _is_memory_request(user_text: str) -> bool:
    txt = (user_text or "").strip().lower()
    if not txt:
        return False
    return any(k in txt for k in _MEMORY_KEYWORDS)


def _format_missing_message() -> str:
    missing = loan_info_memory.missing_required_fields()
    if not missing:
        return ""
    return "계산에 필요한 정보가 아직 부족합니다. 다음 항목을 알려주세요: " + ", ".join(missing)


def _format_result_message() -> str:
    calc = purchase_result_memory._update_result()
    if calc is None:
        return _format_missing_message()
    max_price, max_tax = calc
    return (
        f"최대 매수 가능 아파트 가격: {max_price:,}원\n"
        f"취득세: {max_tax:,}원"
    )


def _format_memory_snapshot() -> str:
    info = loan_info_memory.loan_info
    missing = loan_info_memory.missing_required_fields()

    if missing:
        purchase_result_memory._clear_result()
        status = "대기"
    else:
        calc = purchase_result_memory._update_result()
        status = "완료" if calc is not None else "대기"

    result = purchase_result_memory.result_info

    def _v(x):
        return f"{x:,}원" if isinstance(x, int) else "미입력"

    lines = [
        "[loan_info 메모리]",
        f"- 연소득: {_v(info['annual_income']) if info['annual_income'] is not None else '미입력'}",
        f"- 보유현금: {_v(info['cash']) if info['cash'] is not None else '미입력'}",
        f"- 생애최초구매여부: {info['is_first_home'] if info['is_first_home'] is not None else '미입력'}",
        f"- 보유대출 연이자 상환액: {_v(info['annual_interest_payment']) if info['annual_interest_payment'] is not None else '미입력'}",
        f"- 보유대출 연원금 상환액: {_v(info['annual_principal_payment']) if info['annual_principal_payment'] is not None else '미입력'}",
        "",
        "[purchase_result 메모리]",
        f"- 계산 상태: {status}",
    ]

    if missing:
        lines.append("- 미입력 항목: " + ", ".join(missing))
    else:
        lines.append(f"- max_price: {result['max_price']:,}원")
        lines.append(f"- max_tax: {result['max_tax']:,}원")

    return "\n".join(lines)


_original_run = agent.run


async def _run_with_output_guard(self, message, *args, **kwargs):
    raw = await _original_run(message, *args, **kwargs)

    user_text = str(message) if message is not None else ""
    raw_text = str(raw) if raw is not None else ""

    # 1) 메모리 조회는 LLM 응답 대신 메모리 스냅샷을 강제 반환
    if _is_memory_request(user_text):
        return _format_memory_snapshot()

    # 2) 결과값 요청은 무조건 메모리 계산값으로 응답 고정
    if _is_result_request(user_text):
        forced = _format_result_message()
        return forced if forced else raw

    # 3) 사용자가 결과 요청은 안 했어도 결과 숫자를 임의 제시하면 교정
    if re.search(r"(max_price|max_tax|취득세|최대\s*매수|구매\s*가능\s*아파트\s*가격)", raw_text, re.IGNORECASE):
        missing_msg = _format_missing_message()
        if missing_msg:
            return missing_msg
        return _format_result_message()

    return raw


agent.run = MethodType(_run_with_output_guard, agent)
thread = agent.get_new_thread()
print("✅ Budget Agent 생성 완료! (output guard 적용)")



✅ Budget Agent 생성 완료! (output guard 적용)
